In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import data

In [ ]:
traindata= pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',delimiter="\t")
testdata= pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip',header=0,delimiter="\t")
unlabledata= pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip',header=0,delimiter="\t",quoting=3)
submissoin= pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv',header=0,delimiter="\t")


In [ ]:
traindata['review'][0]

In [ ]:
traindata.head()

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import callbacks, models, layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
import gensim

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words=stopwords.words("english")
def clean(review):
    clean_html=BeautifulSoup(review).get_text()
#     print(clean_html)
    clean_non_letters=re.sub("[^a-zA-Z]"," ",clean_html)
#     print(clean_non_letters)
    cleaned_lowecase=clean_non_letters.lower()
#     print(cleaned_lowecase)
    words=cleaned_lowecase.split()
    clean_words=[w for w in words if w not in stop_words]
    return " ".join(clean_words)

clean("hello my name in 67&%gopal goyal ")

In [ ]:
traindata['cleanreview']=traindata['review'].apply(clean)
testdata['cleanreview']=testdata['review'].apply(clean)

In [ ]:
testdata.head()

# Tokenization

In [ ]:
traindata['tokenized_text']=traindata['cleanreview'].apply(gensim.utils.simple_preprocess)
testdata['tokenized_text']=testdata['cleanreview'].apply(gensim.utils.simple_preprocess)

In [ ]:
traindata.head()

In [ ]:
# handel urls
def remove_url(text):
    url=re.compile(r'https?://S+|www\.\S+')
    return url.sub(r'',text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

# Prediction using count vectorizer

In [ ]:
totaldata=pd.concat([traindata,testdata],axis=0)
totaldata.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
cv=CountVectorizer()

vec=cv.fit_transform(totaldata['cleanreview'])

In [ ]:
X=vec[:len(traindata)]
Y=traindata.sentiment
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1)

# find prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model=RandomForestClassifier()
model.fit(x_train,y_train)
pred=model.predict(x_test)
accuracy_score(pred,y_test)

In [ ]:
print(len(traindata),vec.shape)

In [ ]:
data=vec[len(traindata):]

data.shape

In [ ]:
pred=model.predict(data)
submissoin['sentiment']=pred
submissoin.head()

In [ ]:
submissoin.drop('sentiment',axis=1,inplace=True)
submissoin.columns

In [ ]:
submission= pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv',header=0,delimiter="\t")

In [ ]:
submission['sentiment']=pred

In [ ]:
output = pd.DataFrame(data={"id":testdata.id, "sentiment":pred})
output

In [ ]:
output.to_csv("submission.csv",index=False)